In [111]:
# =====================================================================
# STEP 1 — Imports and Configuration
# =====================================================================

import sagemaker
import boto3
import json
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost import XGBoost
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import TrainingStep
from sagemaker.model_metrics import ModelMetrics, MetricsSource

# Initialize sessions
region = boto3.Session().region_name
pipeline_session = PipelineSession()

sm_client = boto3.client("sagemaker")

# IAM role (automatically retrieved inside SageMaker Studio)
role = sagemaker.get_execution_role()

# Common configuration
project_name = "titanic"
model_package_group_name = "TitanicModel"

# S3 locations
bucket = "ml-pipeline-project-aniolmg"
train_s3 = f"s3://{bucket}/data/titanic_data.csv"
output_s3 = f"s3://{bucket}/models/"

print(f"✅ Using region: {region}")
print(f"✅ Execution role: {role}")
print(f"✅ Output S3: {output_s3}")

✅ Using region: eu-west-3
✅ Execution role: arn:aws:iam::344809604964:role/sage-maker-full-acess-role
✅ Output S3: s3://ml-pipeline-project-aniolmg/models/


In [112]:
# =====================================================================
# STEP 2 — Define and Configure the XGBoost Estimator
# =====================================================================

xgb_estimator = XGBoost(
    entry_point="train_model.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.7-1",
    py_version="py3",
    output_path=f"s3://{bucket}/{project_name}/output/",
    base_job_name=f"{project_name}-train",
    hyperparameters={
        "max_depth": 8,
        "eta": 0.3,
        "objective": "binary:logistic",
        "num_round": 200,
        "bucket": bucket,   # for metric storage in your training script
    },
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.large.


In [113]:
# =====================================================================
# STEP 3 — Create Training, Metrics, and Registration Steps
# =====================================================================

from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.model_metrics import ModelMetrics, MetricsSource
from sagemaker.workflow.step_collections import RegisterModel

# --- Training step (already defined) ---
train_step = TrainingStep(
    name="TrainTitanicModel",
    estimator=xgb_estimator,
    inputs={"train": TrainingInput(train_s3, content_type="csv")},
)

# --- Define a ScriptProcessor for computing metrics ---
script_processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve(
        framework="sklearn",
        region=region,
        version="1.2-1",  # choose a small sklearn image
        py_version="py3",
        image_scope="training"
    ),
    command=["python3"],
    instance_type="ml.t3.medium",  # cheapest option
    instance_count=1,
    role=role,
)

# --- ProcessingStep to compute metrics ---
metrics_step = ProcessingStep(
    name="ComputeTitanicMetrics",
    processor=script_processor,
    inputs=[
        ProcessingInput(
            # Use the training step's model artifacts property (resolved at runtime)
            source=train_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=f"s3://{bucket}/data/",  # dataset
            destination="/opt/ml/processing/data",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="metrics",
            source="/opt/ml/processing/metrics",
            destination=f"s3://{bucket}/metrics/",
        )
    ],
    code="compute_metrics.py",
    depends_on=[train_step],  # ensure metrics run after training
)

# --- Define model metrics using the ProcessingStep output ---
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        # Reference the metrics output dynamically from the ProcessingStep
        s3_uri=metrics_step.properties.ProcessingOutputConfig.Outputs["metrics"].S3Output.S3Uri,
        content_type="application/json",
    )
)

# --- Register the model using the metrics from ProcessingStep ---
register_step = RegisterModel(
    name="RegisterTitanicModel",
    estimator=xgb_estimator,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
    depends_on=[metrics_step],  # ensure registration runs after metrics
)

INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [114]:
# =====================================================================
# STEP 4 — Define and Execute SageMaker Pipeline
# =====================================================================

from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name="TitanicPipeline",
    steps=[train_step, metrics_step, register_step],
    sagemaker_session=pipeline_session,
)

pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


print("✅ Pipeline executed successfully!")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12                                                                                             │
│   13 pipeline.upsert(role_arn=role)                                                              │
│   14 execution = pipeline.start()                                                                │
│ ❱ 15 execution.wait()                                                                            │
│   16                                                                                             │
│   17                                                                                             │
│   18 print("✅ Pipeline executed successfully!")                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline.py:938 in wait               │
│                                                                                                  │
│    935 │   │   waiter = botocore.waiter.create_waiter_with_client(                               │
│    936 │   │   │   waiter_id, model, self.sagemaker_session.sagemaker_client                     │
│    937 │   │   )                                                                                 │
│ ❱  938 │   │   waiter.wait(PipelineExecutionArn=self.arn)                                        │
│    939 │                                                                                         │
│    940 │   def result(self, step_name: str):                                                     │
│    941 │   │   """Retrieves the output of the provided step if it is a ``@step`` decorated func  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/waiter.py:55 in wait                            │
│                                                                                                  │
│    52 │   # Waiter.wait method. This is needed to attach a docstring to the                      │
│    53 │   # method.                                                                              │
│    54 │   def wait(self, **kwargs):                                                              │
│ ❱  55 │   │   Waiter.wait(self, **kwargs)                                                        │
│    56 │                                                                                          │
│    57 │   wait.__doc__ = WaiterDocstring(                                                        │
│    58 │   │   waiter_name=waiter_name,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/waiter.py:374 in wait                           │
│                                                                                                  │
│   371 │   │   │   │   return                                                                     │
│   372 │   │   │   if current_state == 'failure':                                                 │
│   373 │   │   │   │   reason = f'Waiter encountered a terminal failure state: {acceptor.explan   │
│ ❱ 374 │   │   │   │   raise WaiterError(                                                         │
│   375 │   │   │   │   │   name=self.name,                                                        │
│   376 │   │   │   │   │   reason=reason,                                                         │
│   377 │   │   │   │   │   last_response=response,           

In [115]:
import boto3

sm_client = boto3.client("sagemaker")

# Retrieve the latest model package in the group
response = sm_client.list_model_packages(
    ModelPackageGroupName="TitanicModel",
    SortBy="CreationTime",
    SortOrder="Descending",
    MaxResults=1
)

latest_package = response["ModelPackageSummaryList"][0]
print(f"✅ Latest registered model version: {latest_package['ModelPackageVersion']}")
print(f"Model Package ARN: {latest_package['ModelPackageArn']}")
print(f"Approval status: {latest_package['ModelApprovalStatus']}")


✅ Latest registered model version: 16
Model Package ARN: arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/16
Approval status: PendingManualApproval


In [116]:
execution_desc = sm_client.describe_pipeline_execution(
    PipelineExecutionArn=execution.arn
)
print(execution_desc["PipelineExecutionStatus"])


Failed


In [117]:
for step in sm_client.list_pipeline_execution_steps(PipelineExecutionArn=execution.arn)["PipelineExecutionSteps"]:
    print(step["StepName"], step["StepStatus"])


ComputeTitanicMetrics Failed
TrainTitanicModel Succeeded


In [118]:
import boto3

sm_client = boto3.client("sagemaker")

# List all steps in this pipeline execution
steps = sm_client.list_pipeline_execution_steps(
    PipelineExecutionArn=execution.arn
)["PipelineExecutionSteps"]

# Print the status of each step
for step in steps:
    print(f"Step: {step['StepName']}, Status: {step['StepStatus']}")
    if "FailureReason" in step:
        print(f"  Failure reason: {step['FailureReason']}")


Step: ComputeTitanicMetrics, Status: Failed
  Failure reason: ClientError: Failed to invoke sagemaker:CreateProcessingJob. Error Details: No S3 objects found under S3 URL "s3://ml-pipeline-project-aniolmg/titanic/output/pipelines-gmiob8fyeflf-TrainTitanicModel-QrMC0xjt59/output/model.tar.gz" given in input data source. Please ensure that the bucket exists in the selected region (eu-west-3), that objects exist under that S3 prefix, and that the role "arn:aws:iam::344809604964:role/sage-maker-full-acess-role" has "s3:ListBucket" permissions on bucket "ml-pipeline-project-aniolmg".
Step: TrainTitanicModel, Status: Succeeded


In [27]:
# =====================================================================
# STEP 5 — Register Model in the Model Registry (with metrics)
# =====================================================================

# Create a pipeline session (even if you’re running interactively)
pipeline_session = PipelineSession()

# --- Define model metrics object from your S3 JSON ---
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=f"s3://{bucket}/metrics/titanic_metrics.json",
        content_type="application/json"
    )
)

# --- Register the model ---
register_step = RegisterModel(
    name="RegisterTitanicModel",
    estimator=train_step.estimator,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
    role=role,
    sagemaker_session=pipeline_session
)

# Execute the registration step manually
register_step_args = register_step.to_request()
response = sm_client.create_model_package(**register_step_args)

print("✅ Model registered successfully with metrics!")
print("Model Package ARN:", response["ModelPackageArn"])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 )                                                                                           │
│   31                                                                                             │
│   32 # Execute the registration step manually                                                    │
│ ❱ 33 register_step_args = register_step.to_request()                                             │
│   34 response = sm_client.create_model_package(**register_step_args)                             │
│   35                                                                                             │
│   36 print("✅ Model registered successfully with metrics!")                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'RegisterModel' object has no attribute 'to_request'

In [33]:
# --- List registered versions in the Model Registry ---
packages = sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
for pkg in packages["ModelPackageSummaryList"]:
    print(pkg["ModelPackageArn"], "|", pkg["ModelApprovalStatus"])


arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/15 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/14 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/13 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/12 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/11 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/10 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/9 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/8 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/7 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package/TitanicModel/6 | PendingManualApproval
arn:aws:sagemaker:eu-west-3:344809604964:model-package